In [ ]:
import json
import pathlib
import zipfile

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

In [ ]:
import pendulum as pdt
import toolz.curried as toolz

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
    'permian': 'Project_frankNstein_Permian_UTM13_FEET.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
def project_json(path):
    with zipfile.ZipFile(path) as archive:
        return json.loads(archive.read('project.json'))

In [ ]:
bakken_project_json = project_json(project_path_names['bakken'])

In [ ]:
bakken_wells = {w['Name']: w for w in toolz.get_in(['Object', 'Wells'], bakken_project_json)}

In [ ]:
def string_to_date_time(i):
    column_name, value = i
    
    def to_pandas_timestamp(time_text):
        if time_text != '0001-01-01T00:00:00.0000000':
            result = pd.Timestamp(pdt.parse(time_text))
        else:
            result = pd.NaT
        return result
    
    if column_name == 'StartTime':
        return column_name, to_pandas_timestamp(value)
    elif column_name == 'StopTime':
        return column_name, to_pandas_timestamp(value)
    else:
        return column_name, value
    
def stage_details(s):
    result = toolz.pipe(
        s,
        toolz.keyfilter(lambda n: n in {'DisplayStageNumber', 'GlobalStageSequenceNumber', 'StartTime', 'StopTime'}),
        toolz.itemmap(string_to_date_time),
    )
    return result

def stages(project, well):
    result = toolz.pipe(
        toolz.get_in([well, 'Stages'], bakken_wells),
        toolz.map(stage_details),
        toolz.map(lambda s: toolz.merge({'Project': project, 'Well': well}, s)),
        list,
    )
    return result

In [ ]:
# stages('bakken', 'Demo_1H')
# stages('bakken', 'Demo_2H')
# stages('bakken', 'Demo_3H')
# stages('bakken', 'Demo_4H')

In [ ]:
bakken_stages_seq = toolz.concat([
    stages('bakken', 'Demo_1H'),
    stages('bakken', 'Demo_2H'),
    stages('bakken', 'Demo_3H'),
    stages('bakken', 'Demo_4H'),
])

In [ ]:
bakken_stages = pd.DataFrame(data=bakken_stages_seq)

In [ ]:
bakken_stages

In [ ]:
stages_by_seq_no = bakken_stages.set_index('GlobalStageSequenceNumber').sort_index()
stages_by_seq_no

In [ ]:
previous_treatment_starts = stages_by_seq_no.loc[2:, 'StartTime'].to_frame()
previous_treatment_starts.columns = ['PreviousTreatmentStart']
previous_treatment_starts

In [ ]:
stages_with_previous = pd.concat([stages_by_seq_no, previous_treatment_starts], axis=1, copy=False)
stages_with_previous

In [ ]:
stages_with_previous['Switchover'] = (stages_with_previous['StopTime'] - stages_with_previous['PreviousTreatmentStart'])
stages_with_previous

In [ ]:
stages_with_previous['Switchover'] = stages_with_previous['Switchover'].apply(lambda ptd: ptd.total_seconds())
stages_with_previous